In [18]:
import sklearn.linear_model
from sklearn.datasets import fetch_openml

In [19]:
mnist = fetch_openml('mnist_784', version=1)

/Users/zivlazarov/miniforge3/envs/tensorflow-env/lib/python3.8/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [20]:
X, y = mnist['data'], mnist['target']

In [21]:
X.shape

(70000, 784)

In [22]:
X.iloc[0, -3:-1]

pixel782    0.0
pixel783    0.0
Name: 0, dtype: float64

In [23]:
# setting pixel values to be in [0, 1] range
# X = X/255.0

In [24]:
columns = list(X.columns)

In [25]:
new_columns = ['bias']

In [26]:
# new list for the dataframe's columns
new_columns.extend(columns)

In [27]:
# creating the bias column
X['bias'] = 1

In [28]:
X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,bias
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [29]:
# reordering the dataframe with the first column to be the bias
X = X[new_columns]

In [30]:
X.head()

,bias,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# getting the correct shape
X.shape

(70000, 785)

In [32]:
y.shape

(70000,)

In [33]:
y.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [34]:
# working with integers instead of strings for convenience
y = y.apply(lambda x : int(x))

In [35]:
y.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, int64): [0, 1, 2, 3, ..., 6, 7, 8, 9]

In [36]:
import matplotlib.pyplot as plt

In [37]:
import numpy as np


class Perceptron:

    def __init__(self, num_features, num_classes, epochs=1, lr=0.001):
        # num_features here includes the extra bias column with the dataset
        self.num_classes = num_classes
        self.num_features = num_features
        self.num_samples = 0
        self.epochs = epochs
        self.lr = lr
        # initializing the weights vector for every class with values [-1, 1]
        # self.weights = np.random.uniform(low=-1, high=1, size=(num_classes, num_features))
        self.weights = np.random.rand(num_classes, num_features)
        # self.weights = np.zeros(shape=(num_classes, num_features))
        self.final_weights = self.weights
        # determined by the size of samples
        self.labels = None
                  
    def init_multi_class_labels(self, y_train):
        self.num_samples = X_train.shape[0]
        # initializing the perceptron training labels
        self.labels = np.zeros(shape=(self.num_classes, self.num_samples))
        # setting labels' values based on every class in the training data
        for class_idx in range(self.num_classes):
            self.labels[class_idx][y_train == class_idx] = 1
            self.labels[class_idx][y_train != class_idx] = -1

    def fit(self, X_train, y_train):
        self.init_multi_class_labels(y_train)
        # defining error value for each class for pocket algorithm
        min_errors = np.full(shape=(self.num_classes,), fill_value=np.inf)
        # traversing the dataset
        for epoch in range(self.epochs):
            for t in range(self.num_samples):
                # getting the relevant class for this current sample
                class_idx = y_train[t]
                # predicting with the class' weight vector
                y_pred = np.sign(self.weights[class_idx].T.dot(X_train[t]))
                # improving weight vector
                # if y_pred != self.labels[class_idx][t]:
                self.weights[class_idx] += (self.lr * y_pred) * X_train[t]
                # calculating the updated weight vector error
                error_t = np.sum(
                    np.sign(self.weights[class_idx].dot(X_train.T)) != self.labels[class_idx]
                ) / self.num_samples
                # if better than the relevant pocket weight vector, update it
                if error_t < min_errors[class_idx]:
                    self.final_weights[class_idx, :] = self.weights[class_idx, :]
                    min_errors[class_idx] = error_t
    
            
        print(min_errors)
                        
    def predict(self, X_test):
        num_test_samples = len(X_test)
        predictions = np.zeros(shape=(num_test_samples,))
        for i in range(num_test_samples):
            predictions[i] = np.argmax([self.final_weights[idx].T.dot(X_test[i]) for idx in range(self.num_classes)])
        return predictions
        

    def error(self, weights, X, y):
        return np.sum(np.sign(weights.dot(X.T)) != y) / self.num_samples


def sign(value):
    return 1 if value >= 0 else -1

In [38]:
from sklearn.model_selection import train_test_split
num_train_samples = 60000

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=num_train_samples/len(X), shuffle=True)
X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

In [40]:
X_train.shape

(60000, 785)

In [50]:
model = Perceptron(num_features=X_train.shape[1], num_classes=y.nunique(), epochs=10, lr=1)

In [51]:
import time

In [52]:
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

[0.90155    0.88755    0.90065    0.89868333 0.90158333 0.91025
 0.90185    0.89545    0.90188333 0.90055   ]


In [53]:
end_time - start_time

4480.149859189987

In [54]:
predictions = model.predict(X_test)

In [55]:
num_test_samples = len(y_test)
np.sum(predictions == y_test) / num_test_samples

0.6813

In [56]:
with open('model_weights.npy', 'wb') as f:
    np.save(f, model.final_weights)